<a href="https://colab.research.google.com/github/Cyanjiner/classroom-analysis/blob/main/math_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab workspace
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ROOT_FOLDER = "/content/drive/My Drive/transcript_analysis/"
DATA_FOLDER = ROOT_FOLDER + "data/"

RAW_FOLDER = DATA_FOLDER + "raw/"
CLEAN_FOLDER = DATA_FOLDER + "clean/"
MODELS_FOLDER = ROOT_FOLDER + "models/"
LOG_DIR = MODELS_FOLDER + "runs/"

In [24]:
import pandas as pd
from numpy import NaN
import re

# Load all math corpora

- ✅ `ssdf` (`ccss_text.csv`) -- Common Core State Standards

- ✅ `imdf` (`IM_text.csv`) | `math_lang/imdf_sentence_level.csv` -- [Illustrative Mathematics](https://illustrativemathematics.blog/2020/03/26/im-talking-math/)

- ✅ `tdf` (`talk_math.csv`) & `math_lang/talk_math_all.csv | talk_math_sentence.csv`

- `sfdf` (`SFUSD/sfusd_mathtalks.csv`) -- San Francisco Unified School District (SFUSD) Elementary (grade 3-5) Math Talks Bank [orginal resource link](?)

- ✅ `edf` (`engageNY/full_lesson_text.csv`)

- 🤔 (pending for usage) `gdf` (`math_glossary.csv`) -- [CCSS Mathematics Glossary](https://web.archive.org/web/20220206180509/http://www.corestandards.org/Math/Content/mathematics-glossary/)

In [156]:
ssdf = pd.read_csv(DATA_FOLDER + "ccss_text.csv") 
imdf = pd.read_csv(DATA_FOLDER + "IM_text.csv")
tdf = pd.read_csv(DATA_FOLDER + "talk_math.csv")
tdf2 = pd.read_csv(DATA_FOLDER + "tdf2.csv")
gdf = pd.read_csv(DATA_FOLDER + "math_glossary.csv")
sfdf = pd.read_csv(DATA_FOLDER + "SFUSD/sfusd_mathtalks.csv")
edf = pd.read_csv(DATA_FOLDER + "engageNY/full_lesson_text.csv")

## Talking Math

In [763]:
tdf2.head()

,grade,lesson,standard,text
0,K,Invitational 1,NA,"[What kinds of toys do you see?, What can you ..."
1,1,Invitational 1,(K.MD and 1.MD),[How many toys are round? How many have straig...
2,2,Invitational 1,(1.MD and 2.MD.10),"[Complete the sentences:, There are 2 more ___..."
3,3,Invitational 1,(2.MD),"[Pick four kinds of toys and count them. , If ..."
4,4,Invitational 1,3.MD,[These are the lengths of the cars in the pict...


### Clean up Talking Math

In [152]:
import re

In [157]:
with open(DATA_FOLDER + "talking_math.txt", "r") as file:
  tm_text = file.read()

In [167]:
def extract_paren_substring(string):
    start = string.index("(")
    end = string.index(")")
    substring = string[start+1:end]
    return string.replace('('+ substring + ')', ""), substring

In [68]:
extract_paren_substring('(5.NF.B.7) How much juice could all of the oranges in the picture produce?')

(' How much juice could all of the oranges in the picture produce?',
 '5.NF.B.7')

In [168]:
def split_grade_section(lesson_text):
  grade_content = re.split(r"(Grade K+|Grade \d+)", lesson_text)[1:]
  grade_dict = {}
  for j in range(len(grade_content)):
    g_content = grade_content[j]
    if j % 2 == 0: # even index --> grade_content[j] is the separator
      index = grade_content[j]
      #g_text = grade_content[j+1].strip().split('\n')
      g_text = re.sub(r'\s+', ' ', grade_content[j+1])
      g_text = re.sub('Talking Math','', g_text)
      text_list = re.split(r'(?<=[.!?]\s)+',g_text) # split by punctuation characters
      # filtering out empty string & any element w/ "Talking Math" from the g_text list
      text_list = list(filter(None, text_list))
      #get standard if there is one
      for i in range(len(text_list)):
         if  '(' and ')' in text_list[i]:
           new_s, standard = extract_paren_substring(text_list[i])
           text_list[i] = new_s
         else:
           standard = 'NA'
         result = {'standard': standard, "text": [text for text in text_list if text.strip()]}

      grade_dict[index] = result
      #grade_dict[index] = grade_content[j+1]
  return grade_dict

In [169]:
main_section = re.split(r"(Invitational \d+|Day \d+)", tm_text)
lesson = []
text = []
for i, content in enumerate(main_section[1:]):
  if i % 2 != 0: 
    lesson_text = content # if even index --> content is the text follows by lesson
    grade_dict = split_grade_section(lesson_text)
    text.append(grade_dict)
  else: # if odd index --> content is the separator (i.e. lesson index)
    lesson_name = content
    lesson.append(lesson_name)
d = {'lesson': lesson, "text": text}

In [185]:
d['text'][1]

{'Grade 4': {'standard': '3.MD',
  'text': [' These are the lengths of the cars in the picture, organized by color: Turquoise 1 ¾ inch Yellow 1 ¼ inch Red 1 ½ inch Green 1 ¾ inch Orange 1 ¾ inch Describe how you would make a line plot about the length of the cars. ']},
 'Grade 5': {'standard': '4.MD.4',
  'text': [' These are the lengths of the blocks in the picture, organized by color: Turquoise 1 ¼ inch Yellow 1 ½ inch Red 1 ½ inch Green 1 ⅜ inch Orange 1 ½ inch Describe how you would make a line plot of this data about the length of the cars. ']}}

In [183]:
d['text'][1]['Grade 4']['text'][0][83]

'¾'

In [186]:
n = len(d['lesson'])
talk_math_dict = []
for i in range(n):
  lesson_index = d['lesson'][i]
  grade_dict = d['text'][i]
  for grade, values in grade_dict.items():
    grade_name = grade.split(" ")[1]
    standard = values['standard']
    text = values['text']
    talk_math_dict.append({'grade': grade_name, 'lesson':lesson_index, 'standard': standard, 'text': text})

In [189]:
tdf2 = pd.DataFrame(talk_math_dict)

In [190]:
tdf2.head()

,grade,lesson,standard,text
0,K,Invitational 1,NA,"[ What kinds of toys do you see? , What can yo..."
1,1,Invitational 1,K.MD and 1.MD,"[ How many toys are round? , How many have str..."
2,2,Invitational 1,1.MD and 2.MD.10,[ Complete the sentences: There are 2 more ___...
3,3,Invitational 1,2.MD,"[ Pick four kinds of toys and count them. , If..."
4,4,Invitational 1,3.MD,[ These are the lengths of the cars in the pic...


In [200]:
#from google.colab import files
#tdf2.to_csv("tdf2.csv")
#files.download('tdf2.csv')
tdf2.to_csv(DATA_FOLDER + "math_lang/talk_math_all.csv")

### Talking Math -- Sentence Level

In [203]:
tdf2 # filter by grade 3 view

,grade,lesson,standard,text
0,K,Invitational 1,NA,"[ What kinds of toys do you see? , What can yo..."
1,1,Invitational 1,K.MD and 1.MD,"[ How many toys are round? , How many have str..."
2,2,Invitational 1,1.MD and 2.MD.10,[ Complete the sentences: There are 2 more ___...
3,3,Invitational 1,2.MD,"[ Pick four kinds of toys and count them. , If..."
4,4,Invitational 1,3.MD,[ These are the lengths of the cars in the pic...
...,...,...,...,...
644,1,Day 95,1.MD.A.2,"[ Find some of these objects in your home. , H..."
645,2,Day 95,2.MD.A.1,"[ There are rulers in this picture. , What can..."
646,3,Day 95,3.MD.B.4,[ Find 3 of the objects similar to those in th...
647,4,Day 95,4.MD.A.1,[ The longer paint brushes in the picture are ...


In [11]:
import pandas as pd

In [196]:
type(tdf2.iloc[0]['text'])

list

In [201]:
# convert text column to list
#tdf2['text'] = tdf2['text'].apply(lambda x: eval(x))
# split the row into multiple rows using explode
tdf2_by_sentence = tdf2.explode('text')
tdf2_by_sentence.to_csv(DATA_FOLDER + "math_lang/talk_math_sentence_level.csv")

In [202]:
tdf2_by_sentence

,grade,lesson,standard,text
0,K,Invitational 1,NA,What kinds of toys do you see?
0,K,Invitational 1,NA,What can you tell me about the toys?
1,1,Invitational 1,K.MD and 1.MD,How many toys are round?
1,1,Invitational 1,K.MD and 1.MD,How many have straight sides?
1,1,Invitational 1,K.MD and 1.MD,Do you think there are more round toys or toys...
...,...,...,...,...
647,4,Day 95,4.MD.A.1,Can paint brushes be measured in meters?
647,4,Day 95,4.MD.A.1,Explain.
648,5,Day 95,5.MD.A.1,If all the pencils in the picture were lined ...
648,5,Day 95,5.MD.A.1,What is this measure in meters?


## IM text

In [ ]:
imdf.head()

### Decompose by sentence level

In [210]:
imdf['ts_text'] = imdf['ts_text'].apply(lambda x: eval(x))

In [225]:
# decompose by sentence level
imdf_sentence = imdf.explode('ts_text')

In [226]:
imdf_sentence.iloc[0]['ts_text']

'“What are some different ways you and your partner can work together to count the collection?” (One person can count first and then the next can count to see if they get the same amount. We can take turns moving an object and counting a number.)'

In [227]:
imdf_sentence.iloc[3]['ts_text']

'“What does it look and sound like to do math together as a mathematical community?” (We talked to each other and to the teacher. We had quiet time to think. You asked us questions. We shared our ideas. We thought about the math ideas and words we knew. You were writing down our answers. You were waiting quietly until we gave the answers.)'

In [244]:
imdf_sentence.iloc[100]['ts_text']

'“Today we answered questions about data represented with tally marks and numbers. Which representation do you prefer? Why do you like that representation better?” (I prefer tally marks because I don’t have to use cubes or make a drawing to add the numbers together. I prefer the numbers because I don’t have to count.)'

In [245]:
s = re.search(r'“(.*)”\((.*)\)',imdf_sentence.iloc[100]['ts_text'])

In [249]:
double_quotes = re.findall(r'“(.*?)”', imdf_sentence.iloc[100]['ts_text'])

In [252]:
double_quotes

['Today we answered questions about data represented with tally marks and numbers. Which representation do you prefer? Why do you like that representation better?']

In [253]:
parentheses = re.findall(r'\((.*?)\)', imdf_sentence.iloc[100]['ts_text'])

In [264]:
parentheses

['I prefer tally marks because I don’t have to use cubes or make a drawing to add the numbers together. I prefer the numbers because I don’t have to count.']

In [271]:
imdf_sentence['teacher_text']

0        
0        
0        
0        
1        
       ..
2308     
2309     
2309     
2310     
2310     
Name: teacher_text, Length: 12853, dtype: object

### extract teacher & student text respectively into new vars

In [269]:
## extract teacher & student text respectively into new vars
from numpy import NaN
teacher_text = []
student_text = []
for i in range(len(imdf_sentence)):
  ts_text = imdf_sentence.iloc[i]['ts_text']
  if type(ts_text) is str:
    t_text = re.findall(r'“(.*?)”', ts_text)
    s_text = re.findall(r'\((.*?)\)', ts_text)
    if len(t_text) != 0:
      teacher_text.append(t_text[0])
    else:
      teacher_text.append(NaN)
    
    if len(s_text) != 0:
      student_text.append(s_text[0])
    else:
      student_text.append(NaN)
  else:
    teacher_text.append(NaN)
    student_text.append(NaN)
ts_text_dict = {'t_text':teacher_text, 's_text': student_text}

In [277]:
len(ts_text_dict['t_text']) == len(imdf_sentence['ts_text'])

True

In [272]:
t_texts_dt = pd.DataFrame(ts_text_dict)
t_texts_dt.head()

,t_text,s_text
0,What are some different ways you and your part...,One person can count first and then the next c...
1,How did you keep track of the objects as you c...,We put each object in a pile after we counted ...
2,How did they show their count?,They drew twelve circles. They wrote the numbe...
3,What does it look and sound like to do math to...,We talked to each other and to the teacher. We...
4,What are some different ways you and your part...,One person can count first and then the next c...


### Merge with main data

In [289]:
t_texts_dt.reset_index(inplace=True, drop=True)

In [287]:
imdf_sub = imdf_sentence[['grade','lesson','standard','ts_text']] # get a subset by selecting a few columns

In [291]:
imdf_sub.reset_index(inplace=True, drop=True)

In [293]:
imdf_sentence_level = pd.concat([t_texts_dt, imdf_sub],
          axis=1, # axis = 0 binding rows, = 1 bind by cols
          #how = 'inner'
          ignore_index=True) 

In [295]:
imdf_sentence_level.columns = ['index','teacher_text','student_text','grade','lesson','standard','ts_text']

In [297]:
imdf_sentence_level

,index,teacher_text,student_text,grade,lesson,standard,ts_text
0,0,What are some different ways you and your part...,One person can count first and then the next c...,1,Lesson 1,1.OA.C.5,“What are some different ways you and your par...
1,1,How did you keep track of the objects as you c...,We put each object in a pile after we counted ...,1,Lesson 1,1.OA.C.5,“How did you keep track of the objects as you ...
2,2,How did they show their count?,They drew twelve circles. They wrote the numbe...,1,Lesson 1,1.OA.C.5,“How did they show their count?” (They drew tw...
3,3,What does it look and sound like to do math to...,We talked to each other and to the teacher. We...,1,Lesson 1,1.OA.C.5,“What does it look and sound like to do math t...
4,4,What are some different ways you and your part...,One person can count first and then the next c...,1,Lesson 1,1.OA.C.5,“What are some different ways you and your par...
...,...,...,...,...,...,...,...
12848,12848,What were the most important things about your...,I was able to get three different shapes where...,5,Lesson 18,5.MD.C.3,“What were the most important things about you...
12849,12849,What did the writer of this activity have to p...,"the number of sides, the types of angles or co...",5,Lesson 18,5.MD.C.3,“What did the writer of this activity have to ...
12850,12850,What were the most important things about your...,I was able to get three different shapes where...,5,Lesson 18,5.MD.C.3,“What were the most important things about you...
12851,12851,What did the writer of this activity have to p...,"the number of sides, the types of angles or co...",5,Lesson 18,5.MD.C.3,“What did the writer of this activity have to ...


In [298]:
imdf_sentence_level.to_csv(DATA_FOLDER + 'math_lang/imdf_sentence_level.csv')

## Engage NY

In [ ]:
edf

In [310]:
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5873 entries, 0 to 5872
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   5873 non-null   int64 
 1   grade        5873 non-null   object
 2   module       5873 non-null   object
 3   topic        5873 non-null   object
 4   lesson       5873 non-null   int64 
 5   objective    5836 non-null   object
 6   lesson_part  5873 non-null   object
 7   standard     5873 non-null   object
 8   text_type    5873 non-null   object
 9   text         5873 non-null   object
dtypes: int64(2), object(8)
memory usage: 459.0+ KB


In [312]:
edf['grade'].unique()

array(['g4', 'g5', 'g3', 'g6'], dtype=object)

In [314]:
edf['grade'] = edf['grade'].map(lambda x: x.lstrip('g')) # remove all g in grade
# check levels again
edf['grade'].unique()             

array(['4', '5', '3', '6'], dtype=object)

In [329]:
# get a dialogue subset
edf_talk = edf[edf['text_type']=='ts_dialogue'][['grade','standard','lesson_part','text']] 
# reset index
edf_talk.reset_index(inplace=True, drop=True)

In [331]:
# change text variable to list
edf_talk['text'] = edf_talk['text'].apply(lambda x: eval(x))
edf_talk['text'][0]

[['T:',
  '(Project place value chart to the thousands.)  Show 4 ones as place value disks.  Write the number below it.'],
 ['S:', '(Draw 4 ones disks and write 4 below it.) '],
 ['T:', 'Show 4 tens disks, and write the number below it.'],
 ['S:', '(Draw 4 tens disks and write 4 at the bottom of the tens column.)'],
 ['T:', 'Say the number in unit form.'],
 ['S:', '4 tens 4 ones.'],
 ['T:', 'Say the number in standard form.'],
 ['S:', '44.']]

In [341]:
edf_talk_sentence = edf_talk.explode('text')
edf_talk_sentence.reset_index(inplace=True, drop=True)

In [342]:
edf_talk_sentence['text'][0]

['T:',
 '(Project place value chart to the thousands.)  Show 4 ones as place value disks.  Write the number below it.']

In [343]:
edf_talk_sentence['text'][1]

['S:', '(Draw 4 ones disks and write 4 below it.) ']

In [351]:
edf_talk_sentence['text'][1][0][:2]

'S:'

In [348]:
edf_talk_sentence['text'][30258]

['Ms. Johnson and Ms. Siple were folding report cards to send home to parents.  The ratio of the number of report cards Ms. Johnson folded to the number of report cards Ms. Siple folded is .  At the end of the day, Ms. Johnson and Ms. Siple folded a total of  report cards.  How many did each person fold? ']

In [349]:
edf_talk_sentence['text'][30258][0][:2]

'Ms'

In [378]:
edf_talk_sentence['talk_source'] = NaN
edf_talk_sentence['clean_text'] = NaN

In [377]:
edf_talk_sentence

,grade,standard,lesson_part,text,teacher_talk,student_talk,talk_source
0,4,4.NBT.2,Place Value,"[T:, (Project place value chart to the thousan...",(Project place value chart to the thousands.) ...,NaN,NaN
1,4,4.NBT.2,Place Value,"[S:, (Draw 4 ones disks and write 4 below it.) ]",NaN,(Draw 4 ones disks and write 4 below it.),NaN
2,4,4.NBT.2,Place Value,"[T:, Show 4 tens disks, and write the number b...","Show 4 tens disks, and write the number below it.",NaN,NaN
3,4,4.NBT.2,Place Value,"[S:, (Draw 4 tens disks and write 4 at the bot...",NaN,(Draw 4 tens disks and write 4 at the bottom o...,NaN
4,4,4.NBT.2,Place Value,"[T:, Say the number in unit form.]",Say the number in unit form.,NaN,NaN
...,...,...,...,...,...,...,...
30258,6,None,Closing,[Ms. Johnson and Ms. Siple were folding report...,NaN,NaN,NaN
30259,6,None,Closing,"[Ms. Johnson folded report cards, and Ms. Sip...",NaN,NaN,NaN
30260,6,None,Closing,"[At a country concert, the ratio of the number...",NaN,NaN,NaN
30261,6,None,Closing,[There are boys at the country concert. ],NaN,NaN,NaN


### Extract text based on teacher or student & remove instructions

In [383]:
 for i in range(len(edf_talk_sentence)):
  curr_text = edf_talk_sentence['text'][i]
  # start with T:
  if curr_text[0][:2] == 'T:':
    if len(curr_text) < 2 or len(curr_text[1]) <2 :
      continue
    else:
      edf_talk_sentence['talk_source'][i] = 'teacher_text'
      # need to remove every substrings within parentheses --> those are instructions for movement
      if '(' and ')' in curr_text[1]:
        t_talk, instruction = extract_paren_substring(curr_text[1])
        edf_talk_sentence['clean_text'][i] = t_talk
      else:
        edf_talk_sentence['clean_text'][i] = curr_text[1]
  # start with S:
  elif curr_text[0][:2] == 'S:':
    if len(curr_text) < 2 or len(curr_text[1]) <2 :
      continue
    else:
      edf_talk_sentence['talk_source'][i] = 'student_text'
      if '(' and ')' in curr_text[1]:
        t_talk, instruction = extract_paren_substring(curr_text[1])
        edf_talk_sentence['clean_text'][i] = t_talk
      else:
        edf_talk_sentence['clean_text'][i] = curr_text[1]
  # anything else just continue

In [387]:
edf_talk_sentence = edf_talk_sentence.drop(columns=['teacher_talk', 'student_talk'])

In [391]:
edf_talk_sentence['talk_source'].unique()

array(['teacher_text', 'student_text', nan], dtype=object)

In [390]:
edf_talk_sentence.to_csv(DATA_FOLDER + "math_lang/edf_talk_sentence.csv")
files.download(DATA_FOLDER +'math_lang/edf_talk_sentence.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [421]:
edf_talk_sentence

,grade,standard,lesson_part,text,talk_source,clean_text
0,4,4.NBT.2,Place Value,"[T:, (Project place value chart to the thousan...",teacher_text,Show 4 ones as place value disks. Write the...
1,4,4.NBT.2,Place Value,"[S:, (Draw 4 ones disks and write 4 below it.) ]",student_text,
2,4,4.NBT.2,Place Value,"[T:, Show 4 tens disks, and write the number b...",teacher_text,"Show 4 tens disks, and write the number below it."
3,4,4.NBT.2,Place Value,"[S:, (Draw 4 tens disks and write 4 at the bot...",student_text,
4,4,4.NBT.2,Place Value,"[T:, Say the number in unit form.]",teacher_text,Say the number in unit form.
...,...,...,...,...,...,...
30258,6,None,Closing,[Ms. Johnson and Ms. Siple were folding report...,NaN,NaN
30259,6,None,Closing,"[Ms. Johnson folded report cards, and Ms. Sip...",NaN,NaN
30260,6,None,Closing,"[At a country concert, the ratio of the number...",NaN,NaN
30261,6,None,Closing,[There are boys at the country concert. ],NaN,NaN


## SFDF

In [17]:
sfdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    66 non-null     object
 1   grade         66 non-null     object
 2   lesson        66 non-null     object
 3   objective     66 non-null     object
 4   description   66 non-null     object
 5   teacher_talk  62 non-null     object
 6   student_talk  53 non-null     object
dtypes: object(7)
memory usage: 3.7+ KB


In [18]:
# check unique values grade has
sfdf['grade'].unique()

array(['4', 'unit 4', 'unit 5', 'unit 3', 'unit 2', 'grade 4'],
      dtype=object)

In [19]:
sfdf['grade'] = sfdf['grade'].replace(['grade 4','unit 4'], '4')
sfdf['grade'] = sfdf['grade'].replace(['unit 5'], '5')
sfdf['grade'] = sfdf['grade'].replace(['unit 3'], '3')
sfdf['grade'] = sfdf['grade'].replace(['unit 2'], '2')
# check unique values grade has after replacing
sfdf['grade'].unique()

array(['4', '5', '3', '2'], dtype=object)

In [20]:
sfdf['student_talk'][0]

'Anticipated Student Responses:\nI saw 4 dots in a line going down, then 3 dots in a line going down. 4 + 3 is 7, then 2 more dots in a line going down, 2 + 7 = 9. Then the last dot alone is 1, and 1 + 9 = 10.  4 + 3 + 2 + 1 = 10\nI saw a group of 4 in a diamond on the right. Then I saw leftover was 3 dots on the top left and 3 dots on the bottom left. So I added 3 + 3 + 4 which is 6 + 4, which is 10. 3 + 3 + 4 = 10\nI saw a triangle of three dots on the right, the top left, and the bottom left. So I thought of 3 groups of 3 which is 9, then added the one in the middle which is 10.\n3 + 3 + 3 + 1 = 10 or (3 x 3) + 1 = 10'

In [25]:
# remove all the Question/Prompt: | Ask: | Question: from text strings
for i in range(len(sfdf)):
  text = sfdf['teacher_talk'][i]
  if text is not NaN:
    sfdf['teacher_talk'][i] = re.sub(r'(Question/Prompt:\s|Ask:\s|Question:\s)', '', text)

In [26]:
sfdf[['teacher_talk']]

,teacher_talk
0,How many dots do you see? How do you see them?...
1,Description: Ask students Which amount is grea...
2,How many ___ do you see? How do you see them? ...
3,How many ___ do you see? How do you see them? ...
4,Which one doesn’t belong? Why?
...,...
61,What expression might go with this number line...
62,NaN
63,Where does each number go in this diagram?\nAb...
64,Where does each number go in this diagram?\nWi...


## SSDF

In [423]:
ssdf[['text']]

,text
0,Count to 100 by ones and by tens.
1,Count forward beginning from a given number wi...
2,Write numbers from 0 to 20. Represent a number...
3,Understand the relationship between numbers an...
4,"Count to answer ""how many?"" questions about as..."
...,...
326,"Draw construct, and describe geometrical figur..."
327,Solve real-life and mathematical problems invo...
328,Understand congruence and similarity using phy...
329,Understand and apply the Pythagorean Theorem.


# Building Elementary Discourse Math Language Corpus

Pre-processing steps needed:

1. NER tagging & normalization
2. Stopwords removal
3. Lowercase normalization
4. Punctuation removal

### Merging datasets

In [6]:
imdf_sentence_level = pd.read_csv(DATA_FOLDER + "math_lang/imdf_sentence_level.csv") 
talk_math_sentence_level = pd.read_csv(DATA_FOLDER + "math_lang/talk_math_sentence_level.csv") 
edf_talk_sentence = pd.read_csv(DATA_FOLDER + "math_lang/edf_talk_sentence.csv")
sfdf = pd.read_csv(DATA_FOLDER + "SFUSD/sfusd_mathtalks.csv")
ssdf = pd.read_csv(DATA_FOLDER + "ccss_text.csv") 

In [9]:
math_df1_t = imdf_sentence_level[['grade','teacher_text']]
math_df1_s = imdf_sentence_level[['grade','student_text']]

In [27]:
math_df1_t['talk_source'] = 'teacher_text'
math_df1_s['talk_source'] = 'student_text'

<ipython-input-27-8be96c115e83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  math_df1_t['talk_source'] = 'teacher_text'
<ipython-input-27-8be96c115e83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  math_df1_s['talk_source'] = 'student_text'


In [33]:
math_df1_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   grade         12853 non-null  int64 
 1   student_text  12788 non-null  object
 2   talk_source   12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [35]:
math_df1_s.columns = ['grade','text','talk_source']
math_df1_t.columns = ['grade','text','talk_source']

In [36]:
# merge student and teacher text together
math_df1 = pd.concat([math_df1_s, math_df1_t], axis = 0)
math_df1

,grade,text,talk_source
0,1,One person can count first and then the next c...,student_text
1,1,We put each object in a pile after we counted ...,student_text
2,1,They drew twelve circles. They wrote the numbe...,student_text
3,1,We talked to each other and to the teacher. We...,student_text
4,1,One person can count first and then the next c...,student_text
...,...,...,...
12848,5,What were the most important things about your...,teacher_text
12849,5,What did the writer of this activity have to p...,teacher_text
12850,5,What were the most important things about your...,teacher_text
12851,5,What did the writer of this activity have to p...,teacher_text


In [39]:
# check grade level
math_df1['grade'].unique()

array([1, 2, 3, 4, 5])

In [38]:
talk_math_sentence_level['talk_source'] = 'teacher_text'
talk_math_sentence_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1476 non-null   int64 
 1   grade        1476 non-null   object
 2   lesson       1476 non-null   object
 3   standard     1364 non-null   object
 4   text         1476 non-null   object
 5   talk_source  1476 non-null   object
dtypes: int64(1), object(5)
memory usage: 69.3+ KB


In [40]:
# check grade level
talk_math_sentence_level['grade'].unique()

array(['K', '1', '2', '3', '4', '5'], dtype=object)

In [41]:
talk_math_sentence_level['text']

0                         What kinds of toys do you see? 
1                   What can you tell me about the toys? 
2                               How many toys are round? 
3                          How many have straight sides? 
4       Do you think there are more round toys or toys...
                              ...                        
1471            Can paint brushes be measured in meters? 
1472                                            Explain. 
1473     If all the pencils in the picture were lined ...
1474                     What is this measure in meters? 
1475                                           Explain.  
Name: text, Length: 1476, dtype: object

In [44]:
# combine with previous data
math_df2 = pd.concat([math_df1, 
                      talk_math_sentence_level[['grade','text','talk_source']]],
                     axis = 0)

In [45]:
math_df2

,grade,text,talk_source
0,1,One person can count first and then the next c...,student_text
1,1,We put each object in a pile after we counted ...,student_text
2,1,They drew twelve circles. They wrote the numbe...,student_text
3,1,We talked to each other and to the teacher. We...,student_text
4,1,One person can count first and then the next c...,student_text
...,...,...,...
1471,4,Can paint brushes be measured in meters?,teacher_text
1472,4,Explain.,teacher_text
1473,5,If all the pencils in the picture were lined ...,teacher_text
1474,5,What is this measure in meters?,teacher_text


In [43]:
# check grade level
edf_talk_sentence['grade'].unique()
# don't include grade 6 for now
edf_talk_sentence = edf_talk_sentence[edf_talk_sentence['grade'] != 6]
# check grade level again
edf_talk_sentence['grade'].unique()

array([4, 5, 3])

In [49]:
edf_talk_sentence.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30075 entries, 0 to 30074
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   30075 non-null  int64 
 1   grade        30075 non-null  int64 
 2   standard     30075 non-null  object
 3   lesson_part  30075 non-null  object
 4   text         30075 non-null  object
 5   talk_source  23045 non-null  object
 6   clean_text   20503 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.8+ MB


In [62]:
edf_sub = edf_talk_sentence[['grade','clean_text','talk_source']].dropna()

In [63]:
# create a new row storing the length of text
edf_sub['txt_length'] = edf_sub.clean_text.str.len()

# drop rows that len(clean_text) < 5
edf_sub = edf_sub[edf_sub['txt_length'] > 5]

# reset index
edf_sub.reset_index(inplace=True, drop=True)

In [68]:
edf_sub.columns = ['grade','text','talk_source','txt_length']
edf_sub.head()

,grade,text,talk_source,txt_length
0,4,Show 4 ones as place value disks. Write the...,teacher_text,63
1,4,"Show 4 tens disks, and write the number below it.",teacher_text,49
2,4,Say the number in unit form.,teacher_text,28
3,4,4 tens 4 ones.,student_text,14
4,4,Say the number in standard form.,teacher_text,32


In [ ]:
# merge it with math_df2
math_df = pd.concat([math_df2,
                     edf_sub[['grade','text','talk_source']]],
                    axis = 0,
                    ignore_index = True)
math_df

In [72]:
# check string length
math_df['txt_length'] = math_df.text.str.len()

In [74]:
math_df[math_df['txt_length']< 10] 
# well I think these can be considered as noise and remove :)

,grade,text,talk_source,txt_length
159,1,\(6 + 2\,student_text,8.0
167,1,\(6 + 2\,student_text,8.0
175,1,\(6 + 2\,student_text,8.0
183,1,\(6 + 2\,student_text,8.0
292,1,4,student_text,1.0
...,...,...,...,...
46239,3,12 tiles.,student_text,9.0
46241,3,6 twos.,student_text,7.0
46253,3,7 threes.,student_text,9.0
46264,3,4 inches!,student_text,9.0


In [78]:
math_df[math_df['txt_length'] < 15]

,grade,text,talk_source,txt_length
216,1,\(4 + 2 = 6\,student_text,12.0
220,1,\(4 + 2 = 6\,student_text,12.0
224,1,\(4 + 2 = 6\,student_text,12.0
254,1,\(3 + 7 = 10\,student_text,13.0
256,1,\(3 + 4 = 7\,student_text,12.0
...,...,...,...,...
46214,3,Square inches!,student_text,14.0
46231,3,2 × 4 = 1 × 8.,student_text,14.0
46281,3,4 × 7 = 28.,student_text,11.0
46290,3,6 × 8 = 48.,student_text,11.0


In [79]:
math_df3 = math_df[math_df['txt_length'] > 15] 

In [80]:
math_df3[math_df3['txt_length'] < 20]

,grade,text,talk_source,txt_length
417,1,False. \(4 + 2 = 6\,student_text,19.0
422,1,False. \(4 + 2 = 6\,student_text,19.0
427,1,False. \(4 + 2 = 6\,student_text,19.0
519,1,If \(10 + 4 = 14\,student_text,17.0
526,1,If \(10 + 4 = 14\,student_text,17.0
...,...,...,...,...
46112,3,36 square units!,student_text,16.0
46141,3,"5, 7, 8, and 10.",student_text,16.0
46220,3,What is the area?,teacher_text,17.0
46221,3,8 square inches!,student_text,16.0


In [82]:
from google.colab import files
math_df3.to_csv("math_df3.csv")
files.download('math_df3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Remove useless stuff

e.g. 
How do you know?
what did you do?

- incomplete sentences/expressions (maybe)
  - e.g. cross each other

- remove duplicates 18238 duplicates 

- non-math 
  - e.g. how do you know? what did you do?